# 2. Aplicar models

### Cross Validation

In [10]:
import sys

from src import funcionshow
from src import librerias
import numpy as np
import time
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam
from sklearn.model_selection import KFold
from keras.utils import to_categorical


In [12]:
labels,imlabels,imnames,numim,numlab=librerias.crear_labels('../data/dataset/train/')
# Define the model architecture , (64, 240, 240, 3)shape dataset

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(240,240,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))

per poder aplicar un K-Fold Crossvalidation vaig pensar en dividir el train i realitzar un .flow_from_directory() a cada iteracio a partir del train incial que ja te el ImageDataGenerator aplicat.

In [8]:
# Define the K-fold Cross Validator
k_folds = 6
kfold = KFold(n_splits=k_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
print(kfold)

KFold(n_splits=6, random_state=None, shuffle=True)


In [19]:
#necesitem valors numerics per tal de poder aplicar flow
labels = np.array(labels)
aux1 = []
aux2 = []
for i,im in enumerate(imnames):
    image = cv2.imread(im)
    aux1.append(np.array(image.flatten()))
    aux2.append(np.where(labels==imlabels[i])[0][0])
X = np.array(aux1, dtype=object)
y = np.array(aux2)

In [22]:
train=ImageDataGenerator(rescale=1./255, validation_split = 0.1)
acc = []
i =0
for train_index, val_index in kfold.split(X):
    
    
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    print("=========================================")
    print("====== K Fold Validation step => %d/%d =======" % (fold_no,k_folds))
    print("=========================================")
    
    print(X_val)
    
    val_gen = train.flow(x=X_val.flatten(), y=y_val,batch_size=32)
    train_gen = train.flow(x=X_train, y=y_train, batch_size=32) #tienen que ser valores numericos
    
    
    model.fit(generator=train_gen, validation_data=val_gen)
    
    _,accuracy=model.evaluate(val_data, verbose=0)
    fold_no = fold_no + 1
    acc.append(accuracy)
    
print("Average accuracy:", sum(accuracies) / len(accuracies))

====== K Fold Validation step => 1/6 =======
[array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)
 array([0, 0, 0, ..., 0, 0, 0], dtype=uint8) ...
 array([130, 128, 126, ..., 215, 205, 199], dtype=uint8)
 array([236, 232, 236, ..., 247, 249, 251], dtype=uint8)
 array([255, 255, 255, ..., 255, 255, 255], dtype=uint8)]


ValueError: setting an array element with a sequence.